In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/SloganGenerator/venv/lib/python3.10/site-packages')

# Create the dataset

In [ ]:
import transformers
from datasets import load_dataset, load_metric

In [ ]:
slogan_dataset_raw = load_dataset("/content/drive/MyDrive/SloganGenerator/dataset/", data_files="merged.csv")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/dataset-81ac2a11ea7f9045/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
print(slogan_dataset_raw)

DatasetDict({
    train: Dataset({
        features: ['company', 'slogan'],
        num_rows: 11902
    })
})


In [ ]:
slogan_dataset = slogan_dataset_raw["train"].train_test_split(train_size=0.9, seed=20)
slogan_dataset["validation"] = slogan_dataset.pop("test")
print(slogan_dataset)

DatasetDict({
    train: Dataset({
        features: ['company', 'slogan'],
        num_rows: 10711
    })
    validation: Dataset({
        features: ['company', 'slogan'],
        num_rows: 1191
    })
})


In [ ]:
import nltk
nltk.download('punkt')
import string
from transformers import AutoTokenizer

MAX_LENGTH = 64
model_checkpoint = "t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
slogan_dataset = slogan_dataset.filter(
    lambda example: (example['company'] != None) and
    (example['slogan'] != None)
)

Filter:   0%|          | 0/10711 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1191 [00:00<?, ? examples/s]

In [ ]:
def preprocess_data(examples):
  inputs = [ex for ex in examples['company']]
  model_inputs = tokenizer(inputs, max_length=MAX_LENGTH, truncation=True)

  # Setup the tokenizer for targets
  with tokenizer.as_target_tokenizer():
    labels = tokenizer(examples["slogan"], max_length=MAX_LENGTH, 
                       truncation=True)

  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

In [ ]:
tokenized_dataset = slogan_dataset.map(preprocess_data, batched=True, remove_columns=slogan_dataset["train"].column_names)

print(tokenized_dataset['train'])

Map:   0%|          | 0/10711 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3596: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1191 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 10711
})


# Instantiate the model

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

batch_size = 8
model_name = "t5_merged"
model_dir = f"/content/drive/MyDrive/SloganGenerator/models/{model_name}"

In [ ]:
args = Seq2SeqTrainingArguments(
    model_dir,
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    learning_rate=4e-5,
    per_device_train_batch_size=batch_size*4,
    per_device_eval_batch_size=batch_size*8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=6,
    predict_with_generate=True,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="rouge1"
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer)

In [ ]:

metric = load_metric("rouge")

In [ ]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip()))
                      for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) 
                      for label in decoded_labels]
    
    # Compute ROUGE scores
    result = metric.compute(predictions=decoded_preds, references=decoded_labels,
                            use_stemmer=True)

    # Extract ROUGE f1 scores
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length to metrics
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id)
                      for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
def model_init():
    return AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

trainer = Seq2SeqTrainer(
    model_init=model_init,
    args=args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Training

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,3.270700,2.751844,17.938500,6.745800,17.307300,17.366700,8.524800
2,2.840300,2.679034,18.498200,6.842000,17.661700,17.847000,8.916000
3,2.733000,2.644787,18.239100,6.802700,17.451300,17.607700,8.753100
4,2.658200,2.625421,18.269300,7.027400,17.502200,17.688000,8.740600
5,2.614600,2.617494,18.303800,6.985400,17.540100,17.692800,8.849700
6,2.588100,2.614016,18.204600,6.924300,17.440200,17.612900,8.825400


TrainOutput(global_step=2010, training_loss=2.784151902839319, metrics={'train_runtime': 658.636, 'train_samples_per_second': 97.574, 'train_steps_per_second': 3.052, 'total_flos': 3440732208122880.0, 'train_loss': 2.784151902839319, 'epoch': 6.0})

# Testing

In [ ]:
import tqdm

In [ ]:
model_name = "t5_merged/checkpoint-2010"
model_dir = f"/content/drive/MyDrive/SloganGenerator/models/{model_name}"

tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)

max_input_length = 64

In [ ]:
inputs = ["Olivetti is an Italian manufacturer of computers, tablets, smartphones, printers and other such business products as calculators and fax machines."]

inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, return_tensors="pt")

slogan = []

for i in tqdm.tqdm(range(20)):
  output = model.generate(**inputs, num_beams=1, do_sample=True, min_length=10, max_length=64)
  decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
  slogan.append(decoded_output)

print()
for s in slogan:
  print(s)

100%|██████████| 20/20 [00:15<00:00,  1.29it/s]


Olivetti. The Italian innovation for business
Olivetti. The Smart Company in IT
Olivetti & Co. Ltd.
Olivetti. The world's original fax maker.
Olivetti. A smart world without a desk
Olivetti is the Italian business to beat
Olivetti. Where everything is Italian
Olivetti. Technology for business.
Olivetti. Your choice of solutions
Olivetti. The world's first printer
Olivetti. The best choice for IT industry
Olivetti, the computer giant with roots in Italy
Business world leader. The world market leader.
Olivetti. Easy solutions. Fast results.
Olivetti. Taking care of you - Your Business
Olivetti. Creating jobs.
Olivetti - Quality over quantity
Olivetti. The best printing home in the world
Olivetti. Think & Work
Olivetti is a good looking brand
